**Set environment**

In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Script**

In [2]:
cat > run_annot_test.sh << 'EOF'
#!/bin/bash

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### setup input and output
source ../config/config_duke.sh
FP_REG=$1
FP_ANT=$2
FP_OUT=$3

### show I/O file
echo "Input: " ${FP_REG}
echo
echo "show first few lines of input"
zcat ${FP_REG} | head -5
echo
echo "Input: " ${FP_ANT}
echo
echo "show first few lines of input"
zcat ${FP_ANT} | head -5
echo

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

chmod +x run_annot_test.sh

In [3]:
cat > run_annot.sh << 'EOF'
#!/bin/bash

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### setup input and output
source ../config/config_duke.sh
FP_REG=$1
FP_ANT=$2
FP_OUT=$3

### show I/O file
echo "Input: " ${FP_REG}
echo
echo "show first few lines of input"
zcat ${FP_REG} | head -5
echo
echo "Input: " ${FP_ANT}
echo
echo "show first few lines of input"
zcat ${FP_ANT} | head -5
echo

### execute: annotation using intersect
bedtools intersect \
    -a ${FP_REG} \
    -b ${FP_ANT} \
    -wo \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

chmod +x run_annot.sh

## ATAC peaks

In [4]:
FD_REG=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz
FP_REG=${FD_REG}/${FN_REG}
zcat ${FP_REG} | head

chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613
chr1	30803	31072
chr1	101603	101849
chr1	115411	115986
chr1	118518	118743
chr1	136071	137429


## CRISPRi-FlowFISH

**Check data**

In [5]:
ASSAY=CRISPRi_FlowFISH
FD_INP=${FD_RES}/results/${ASSAY}
ls ${FD_INP}

coverage  coverage_astarrseq_peak_macs_input


In [6]:
ASSAY=CRISPRi_FlowFISH
FD_ANT=${FD_RES}/results/${ASSAY}/coverage
FN_ANT=Merge_HCRFF_rAVG.log2FC_2.filtered.bed.gz
FP_ANT=${FD_ANT}/${FN_ANT}
zcat ${FP_ANT} | head

chr11	33064196	33064197	id-1495	0.807624	CAPRIN1
chr11	33064217	33064218	id-1496	-2.71246	CAPRIN1
chr11	33064229	33064230	id-1497	0.642045	CAPRIN1
chr11	33064230	33064231	id-1498	1.85354	CAPRIN1
chr11	33064258	33064259	id-1499	0.126467	CAPRIN1
chr11	33064264	33064265	id-1500	0.217096	CAPRIN1
chr11	33064298	33064299	id-1501	-0.612031	CAPRIN1
chr11	33064303	33064304	id-1502	-0.859612	CAPRIN1
chr11	33064315	33064316	id-1503	-0.665188	CAPRIN1
chr11	33064339	33064340	id-1504	-0.569748	CAPRIN1


**RUN**

In [7]:
### ATAC peaks
FD_REG=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz
FP_REG=${FD_REG}/${FN_REG}

### Annotated signals
ASSAY=CRISPRi_FlowFISH
FD_ANT=${FD_RES}/results/${ASSAY}/coverage
FN_ANT=Merge_HCRFF_rAVG.log2FC_2.filtered.bed.gz
FP_ANT=${FD_ANT}/${FN_ANT}

### Output file
FD_OUT=${FD_RES}/results/${ASSAY}/coverage_astarrseq_peak_macs_input
FN_OUT=${FN_ANT}
FP_OUT=${FD_OUT}/${FN_OUT}

### Log file
FN_LOG=coverage_astarr_peak_macs_input_crispri_hcrff.%a.txt
FP_LOG=${FD_LOG}/${FN_LOG}
echo '${FD_LOG}/coverage_astarr_peak_macs_input_crispri_hcrff.0.txt'

### RUN
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 4 \
    --mem 4G \
    --output ${FP_LOG} \
    --array 0 \
    ./run_annot.sh ${FP_REG} ${FP_ANT} ${FP_OUT}

${FD_LOG}/coverage_astarr_peak_macs_input_crispri_hcrff.0.txt
Submitted batch job 29958339


**Check results**

In [8]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_crispri_hcrff.0.txt

Hostname:           x1-03-1.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         06-10-23+17:14:49

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/CRISPRi_FlowFISH/coverage/Merge_HCRFF_rAVG.log2FC_2.filtered.bed.gz

show first few lines of input
chr11	33064196	33064197	id-1495	0.807624	CAPRIN1
chr11	33064217	33064218	id-1496	-2.71246	CAPRIN1
chr11	33064229	33064230	id-1497	0.642045	CAPRIN1
chr11	33064230	33064231	id-1498	1.85354	CAPRIN1
chr11	33064258	33064259	id-1499	0.126467	CAPRIN1


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/CRISPRi_FlowFISH/coverage_astarrseq_peak_macs_input/Merge_HCRFF_rAVG.log2FC_2.filtered.bed.gz

show first few line

## CRISPRi-Growth

**Check data**

In [9]:
ASSAY=CRISPRi_Growth_K562_Gersbach
FD_ANT=${FD_RES}/results/${ASSAY}/coverage
ls -1 ${FD_ANT}

CRISPRi_rAVG_log2FC_2.bed
CRISPRi_rAVG_log2FC_2.bed.gz
CRISPRi_rAVG_log2FC_2.wig


In [10]:
ASSAY=CRISPRi_Growth_K562_Gersbach
FD_ANT=${FD_RES}/results/${ASSAY}/coverage
FN_ANT=CRISPRi_rAVG_log2FC_2.bed.gz
FP_ANT=${FD_ANT}/${FN_ANT}
zcat ${FP_ANT} | head

chr1	605580	605581	id-1	-0.180660
chr1	605581	605582	id-2	0.486646
chr1	605589	605590	id-3	3.171240
chr1	605591	605592	id-4	-0.364573
chr1	605592	605593	id-5	-0.206319
chr1	605609	605610	id-6	-0.266750
chr1	605617	605618	id-7	-0.774100
chr1	605618	605619	id-8	-0.353224
chr1	605619	605620	id-9	-0.187707
chr1	605620	605621	id-10	-0.420401


**RUN**

In [11]:
### ATAC peaks
FD_REG=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz
FP_REG=${FD_REG}/${FN_REG}

### Annotated signals
ASSAY=CRISPRi_Growth_K562_Gersbach
FD_ANT=${FD_RES}/results/${ASSAY}/coverage
FN_ANT=CRISPRi_rAVG_log2FC_2.bed.gz
FP_ANT=${FD_ANT}/${FN_ANT}

### Output file
FD_OUT=${FD_RES}/results/${ASSAY}/coverage_astarrseq_peak_macs_input
FN_OUT=${FN_ANT}
FP_OUT=${FD_OUT}/${FN_OUT}

### Log file
FN_LOG=coverage_astarr_peak_macs_input_crispri_growth.%a.txt
FP_LOG=${FD_LOG}/${FN_LOG}
echo '${FD_LOG}/coverage_astarr_peak_macs_input_crispri_growth.0.txt'

### RUN
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 4 \
    --mem 4G \
    --output ${FP_LOG} \
    --array 0 \
    ./run_annot.sh ${FP_REG} ${FP_ANT} ${FP_OUT}

${FD_LOG}/coverage_astarr_peak_macs_input_crispri_growth.0.txt
Submitted batch job 29958340


**Check results**

In [12]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_crispri_growth.0.txt

Hostname:           x1-03-1.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         06-10-23+17:15:02

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/CRISPRi_Growth_K562_Gersbach/coverage/CRISPRi_rAVG_log2FC_2.bed.gz

show first few lines of input
chr1	605580	605581	id-1	-0.180660
chr1	605581	605582	id-2	0.486646
chr1	605589	605590	id-3	3.171240
chr1	605591	605592	id-4	-0.364573
chr1	605592	605593	id-5	-0.206319


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/CRISPRi_Growth_K562_Gersbach/coverage_astarrseq_peak_macs_input/CRISPRi_rAVG_log2FC_2.bed.gz

show first few lines of output:
chr1	605090	605823	chr1	605580	605581	id-1	-0.180660	1
chr1	60509

## LentiMPRA

**Check data**

In [5]:
ASSAY=LentiMPRA_K562_Nadav
FD_ANT=${FD_RES}/results/${ASSAY}/coverage
ls -1 ${FD_ANT}

K562.ENCSR382BVV.ENCFF348WYK.LMPRA.region.bed.gz
K562.ENCSR382BVV.ENCFF348WYK.LMPRA.region_merge.bed.gz
K562.ENCSR382BVV.ENCFF348WYK.LMPRA.region_merge_perbase.bed.gz
K562.ENCSR382BVV.ENCFF348WYK.LMPRA.stranded.bed.gz
K562.ENCSR382BVV.ENCFF348WYK.LMPRA.unstranded.bed.gz
K562.ENCSR382BVV.ENCFF348WYK.LMPRA.unstranded.bg
K562.ENCSR382BVV.ENCFF348WYK.LMPRA.unstranded.bw
K562.vikram230621.LMPRA.region.bed.gz
K562.vikram230621.LMPRA.region_merge.bed.gz
K562.vikram230621.LMPRA.region_merge_perbase.bed.gz
K562.vikram230621.LMPRA.stranded.bed.gz
K562.vikram230621.LMPRA.unstranded.bg
K562.vikram230621.LMPRA.unstranded.bw
nadav_K562_MPRA_log2FC_Z_bp.bed
nadav_K562_MPRA_log2FC_Z_bp.bed.gz
nadav_K562_MPRA_log2FC_Z_bp.wig


In [6]:
ASSAY=LentiMPRA_K562_Nadav
FD_ANT=${FD_RES}/results/${ASSAY}/coverage
FN_ANT=nadav_K562_MPRA_log2FC_Z_bp.bed.gz
FP_ANT=${FD_ANT}/${FN_ANT}
zcat ${FP_ANT} | head

chr1	778637	778638	id-1	2.176040
chr1	778638	778639	id-2	2.176040
chr1	778639	778640	id-3	2.176040
chr1	778640	778641	id-4	2.176040
chr1	778641	778642	id-5	2.176040
chr1	778642	778643	id-6	2.176040
chr1	778643	778644	id-7	2.176040
chr1	778644	778645	id-8	2.176040
chr1	778645	778646	id-9	2.176040
chr1	778646	778647	id-10	2.176040


In [7]:
ASSAY=LentiMPRA_K562_Nadav
FD_ANT=${FD_RES}/results/${ASSAY}/coverage
FN_ANT=K562.ENCSR382BVV.ENCFF348WYK.LMPRA.stranded.bed.gz
FP_ANT=${FD_ANT}/${FN_ANT}
zcat ${FP_ANT} | head

chr1	115631	115831	peak1_Reversed:	0.1572	-
chr1	138940	139140	peak2_Reversed:	0.2047	-
chr1	138940	139140	peak2	-0.2317	+
chr1	203225	203425	peak3_Reversed:	0.0023	-
chr1	203225	203425	peak3	0.0524	+
chr1	267911	268111	peak4_Reversed:	-0.322	-
chr1	267911	268111	peak4	0.4415	+
chr1	586093	586293	peak5_Reversed:	0.0718	-
chr1	586093	586293	peak5	0.081	+
chr1	629847	630047	peak6_Reversed:	-0.7038	-


In [8]:
ASSAY=LentiMPRA_K562_Nadav
FD_ANT=${FD_RES}/results/${ASSAY}/coverage
FN_ANT=K562.vikram230621.LMPRA.stranded.bed.gz
FP_ANT=${FD_ANT}/${FN_ANT}
zcat ${FP_ANT} | head

chr1	115631	115831	peak1_Reversed:	0.04	-	potential enhancer
chr1	138940	139140	peak2	-0.543	+	potential enhancer
chr1	138940	139140	peak2_Reversed:	-0.101	-	potential enhancer
chr1	203225	203425	peak3	-0.186	+	potential enhancer
chr1	203225	203425	peak3_Reversed:	-0.281	-	potential enhancer
chr1	267911	268111	peak4	0.225	+	potential enhancer
chr1	267911	268111	peak4_Reversed:	-0.967	-	potential enhancer
chr1	586093	586293	peak5	-0.441	+	potential enhancer
chr1	586093	586293	peak5_Reversed:	-0.108	-	potential enhancer
chr1	629847	630047	peak6	-0.562	+	potential enhancer


**RUN**

In [15]:
### ATAC peaks
FD_REG=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz
FP_REG=${FD_REG}/${FN_REG}

### Annotated signals
ASSAY=LentiMPRA_K562_Nadav
FD_ANT=${FD_RES}/results/${ASSAY}/coverage
FN_ANT=nadav_K562_MPRA_log2FC_Z_bp.bed.gz
FP_ANT=${FD_ANT}/${FN_ANT}

### Output file
FD_OUT=${FD_RES}/results/${ASSAY}/coverage_astarrseq_peak_macs_input
FN_OUT=${FN_ANT}
FP_OUT=${FD_OUT}/${FN_OUT}

### Log file
FN_LOG=coverage_astarr_peak_macs_input_lmpra.%a.txt
FP_LOG=${FD_LOG}/${FN_LOG}
echo '${FD_LOG}/coverage_astarr_peak_macs_input_lmpra.0.txt'

### RUN
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 4 \
    --mem 4G \
    --output ${FP_LOG} \
    --array 0 \
    ./run_annot.sh ${FP_REG} ${FP_ANT} ${FP_OUT}

${FD_LOG}/coverage_astarr_peak_macs_input_lmpra.0.txt
Submitted batch job 29958341


In [8]:
### ATAC peaks
FD_REG=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz
FP_REG=${FD_REG}/${FN_REG}

### Annotated signals
ASSAY=LentiMPRA_K562_Nadav
FD_ANT=${FD_RES}/results/${ASSAY}/coverage
FN_ANT=K562.ENCSR382BVV.ENCFF348WYK.LMPRA.stranded.bed.gz
FP_ANT=${FD_ANT}/${FN_ANT}

### Output file
FD_OUT=${FD_RES}/results/${ASSAY}/coverage_astarrseq_peak_macs_input
FN_OUT=${FN_ANT}
FP_OUT=${FD_OUT}/${FN_OUT}

### Log file
FN_LOG=coverage_astarr_peak_macs_input_lmpra_v2.%a.txt
FP_LOG=${FD_LOG}/${FN_LOG}
echo '${FD_LOG}/coverage_astarr_peak_macs_input_lmpra_v2.0.txt'

### RUN
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 4 \
    --mem 4G \
    --output ${FP_LOG} \
    --array 0 \
    ./run_annot.sh ${FP_REG} ${FP_ANT} ${FP_OUT}

${FD_LOG}/coverage_astarr_peak_macs_input_lmpra_v2.0.txt
Submitted batch job 30021630


In [12]:
### ATAC peaks
FD_REG=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz
FP_REG=${FD_REG}/${FN_REG}

### Annotated signals
ASSAY=LentiMPRA_K562_Nadav
FD_ANT=${FD_RES}/results/${ASSAY}/coverage
FN_ANT=K562.vikram230621.LMPRA.stranded.bed.gz
FP_ANT=${FD_ANT}/${FN_ANT}

### Output file
FD_OUT=${FD_RES}/results/${ASSAY}/coverage_astarrseq_peak_macs_input
FN_OUT=${FN_ANT}
FP_OUT=${FD_OUT}/${FN_OUT}

### Log file
FN_LOG=coverage_astarr_peak_macs_input_lmpra_v3_vikram230621.%a.txt
FP_LOG=${FD_LOG}/${FN_LOG}
echo '${FD_LOG}/coverage_astarr_peak_macs_input_lmpra_v3_vikram230621.0.txt'

### RUN
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 4 \
    --mem 4G \
    --output ${FP_LOG} \
    --array 0 \
    ./run_annot.sh ${FP_REG} ${FP_ANT} ${FP_OUT}

${FD_LOG}/coverage_astarr_peak_macs_input_lmpra_v3_vikram230621.0.txt
Submitted batch job 30021944


**Check results**

In [11]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_lmpra.0.txt

Hostname:           x1-03-1.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         06-10-23+17:15:29

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/LentiMPRA_K562_Nadav/coverage/nadav_K562_MPRA_log2FC_Z_bp.bed.gz

show first few lines of input
chr1	778637	778638	id-1	2.176040
chr1	778638	778639	id-2	2.176040
chr1	778639	778640	id-3	2.176040
chr1	778640	778641	id-4	2.176040
chr1	778641	778642	id-5	2.176040


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/LentiMPRA_K562_Nadav/coverage_astarrseq_peak_macs_input/nadav_K562_MPRA_log2FC_Z_bp.bed.gz

show first few lines of output:
chr1	777949	779437	chr1	778637	778638	id-1	2.176040	1
chr1	777949	779437

In [9]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_lmpra_v2.0.txt

Hostname:           x1-01-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         06-20-23+20:37:33

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/LentiMPRA_K562_Nadav/coverage/K562.ENCSR382BVV.ENCFF348WYK.LMPRA.stranded.bed.gz

show first few lines of input
chr1	115631	115831	peak1_Reversed:	0.1572	-
chr1	138940	139140	peak2_Reversed:	0.2047	-
chr1	138940	139140	peak2	-0.2317	+
chr1	203225	203425	peak3_Reversed:	0.0023	-
chr1	203225	203425	peak3	0.0524	+


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/LentiMPRA_K562_Nadav/coverage_astarrseq_peak_macs_input/K562.ENCSR382BVV.ENCFF348WYK.LMPRA.stranded.bed.gz

show first few lines of output:
chr1

In [13]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_lmpra_v3_vikram230621.0.txt

Hostname:           x1-02-1.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         06-23-23+14:12:07

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/LentiMPRA_K562_Nadav/coverage/K562.vikram230621.LMPRA.stranded.bed.gz

show first few lines of input
chr1	115631	115831	peak1_Reversed:	0	-	potential enhancer	0.138	-0.044	0.025	0.04	2.4069025995500155	1.746127748956286	1.9966412912143483	2.0511007569226227
chr1	138940	139140	peak2_Reversed:	0	-	potential enhancer	-0.37	0.33	-0.261	-0.101	0.5625420275631225	3.103983760615928	0.9582808117099164	1.5391817792648435
chr1	138940	139140	peak2	0	+	potential enhancer	-0.245	-0.609	-0.775	-0.543	1.0163709084654091	-0.3051787927220